In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''


import sys

sys.path.append('/notebook/framebank_parser/roleinduction/ri_tools/src/liash')
sys.path.append('/notebook/framebank_parser/roleinduction/pylingtools/src/')
sys.path.append('./isanlp/src/')

In [5]:
import os
import time
import pickle
import isanlp
import json

In [6]:
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd
from isanlp import PipelineCommon
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.ru.processor_tokenizer_ru import ProcessorTokenizerRu
from isanlp.processor_sentence_splitter import ProcessorSentenceSplitter
from isanlp.processor_syntaxnet_remote import ProcessorSyntaxNetRemote
from isanlp import PipelineCommon

# Parse framebank

In [3]:
from isanlp.wrapper_multi_process_document import WrapperMultiProcessDocument


ppl = WrapperMultiProcessDocument(
    [
        PipelineCommon([(ProcessorRemote('192.168.0.104', 3333, 'default'), 
                         ['text'], 
                         {'sentences' : 'sentences', 
                          'tokens' : 'tokens',
                          'postag' : 'mystem_postags',
                          'lemma' : 'lemma'}),
                        (ProcessorSyntaxNetRemote(addr, port), 
                         ['tokens', 'sentences'], 
                         {'syntax_dep_tree' : 'syntax_dep_tree'}),
                        (ConverterMystemToUd(), 
                         ['mystem_postags'], 
                         {0 : 'postag'})])
        for addr, port in [('exn40.isa.ru', 8200), 
                           ('exn40.isa.ru', 8201),
                           ('exn40.isa.ru', 8202),
                           ('exn40.isa.ru', 8203),
                           ('exn40.isa.ru', 8204),
                           ('exn40.isa.ru', 8205),
                           ('exn40.isa.ru', 8206),
                           ('exn40.isa.ru', 8207),
                           ('exn40.isa.ru', 8208),
                           ('exn40.isa.ru', 8209),
                           
                           #('exn40.isa.ru', 8113),
                           ('vmh1.isa.ru', 8200),
                           ('vmh1.isa.ru', 8201),
                           ('vmh1.isa.ru', 8202),
                           ('vmh1.isa.ru', 8203),
                           ('vmh1.isa.ru', 8204),
                           
                           ('vmh2.isa.ru', 8200),
                           ('vmh2.isa.ru', 8201),
                           ('vmh2.isa.ru', 8202),
                           ('vmh2.isa.ru', 8203),
                           ('vmh2.isa.ru', 8204),
                           ('vmh2.isa.ru', 8205),
                           ('vmh2.isa.ru', 8206),
                           ('vmh2.isa.ru', 8207),
                           ('vmh2.isa.ru', 8208),
                           
                           ('192.168.0.104', 8200),
                           ('192.168.0.104', 8201)]
    ]
)

ppl(['Мама мыла раму.', 'Папа чинил машину.'])    

100%|██████████| 2/2 [00:01<00:00,  1.17it/s]


[{'lemma': [['мама', 'мыть', 'рама', '.']],
  'mystem_postags': [['S,жен,од=им,ед',
    'V,несов,пе=прош,ед,изъяв,жен',
    'S,жен,неод=вин,ед',
    '']],
  'postag': [[{'Animacy': 'Anim',
     'Case': 'Nom',
     'Gender': 'Fem',
     'Number': 'Sing',
     'fPOS': 'NOUN'},
    {'Aspect': 'Imp',
     'Gender': 'Fem',
     'Number': 'Sing',
     'Tense': 'Past',
     'Valency': 'TR',
     'VerbForm': 'Fin',
     'fPOS': 'VERB'},
    {'Animacy': 'Inan',
     'Case': 'Acc',
     'Gender': 'Fem',
     'Number': 'Sing',
     'fPOS': 'NOUN'},
    {}]],
  'sentences': [<isanlp.annotation.Sentence at 0x7f0e1d9999e8>],
  'syntax_dep_tree': [[<isanlp.annotation.WordSynt at 0x7f0e1d999d30>,
  'text': 'Мама мыла раму.',
  'tokens': [<isanlp.annotation.Token at 0x7f0e1d999ba8>,
   <isanlp.annotation.Token at 0x7f0e1d999c18>]},
 {'lemma': [['папа', 'чинить', 'машина', '.']],
  'mystem_postags': [['S,муж,од=им,ед',
    'V,несов,пе=прош,ед,изъяв,муж',
    'S,жен,неод=вин,ед',
    '']],
  'postag': [[

In [4]:
input_data_path = 'data/cleared_corpus.json'

with open(input_data_path, 'r') as f:
    data = json.load(f)
    
print('Number of examples: ', len(data))

Number of examples:  32612


In [5]:
from isanlp.wrapper_multi_process_document import split_equally
data_pts = split_equally(data, 8)

In [6]:
from convert_corpus_to_brat import make_text, create_verb_example_index

result_pts = []

for data_pt in data_pts:
    texts = [make_text(example, 0)[0] for (ex_id, example) in data_pt]
    result_pt = ppl(texts)
    result_pts.append(result_pt)

100%|██████████| 4077/4077 [08:55<00:00,  7.61it/s]


In [7]:
with open('data/results-1.pckl', 'wb') as f:
    pickle.dump(result_pts, f)

In [18]:
final_res = []
for res in result_pts:
    final_res += res

final_res_with_ex_id = [(data[i][0], e) for (i, e) in enumerate(final_res)]

with open('data/results_final.pckl', 'wb') as f:
    pickle.dump(final_res_with_ex_id, f)

# Fix issues with morph and postag

In [19]:
with open('data/results_final.pckl', 'rb') as f:
    ling_annots_to_fix = pickle.load(f)

In [20]:
for ex_id, ling_annot in ling_annots_to_fix:
    ling_annot['morph'] = []
    for i in range(len(ling_annot['lemma'])):
        sent = ling_annot['postag'][i]
        ling_annot['morph'].append([])
        for j in range(len(sent)):
            ling_annot['morph'][i].append(ling_annot['postag'][i][j])
            ling_annot['postag'][i][j] = ling_annot['morph'][i][j].get('fPOS', '')

In [23]:
with open('data/results_final_fixed.pckl', 'wb') as f:
    pickle.dump(ling_annots_to_fix, f)